In [9]:
from time import sleep
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import os
from selenium.webdriver.support.ui import WebDriverWait
import requests
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.common.exceptions import TimeoutException
import datetime
from datetime import datetime, timedelta
from tqdm import tqdm

In [10]:
def scrape_kayak(from_location, to_location, date):

    formatted_date = date.strftime('%Y-%m-%d')
    url = f'https://www.kayak.com/flights/{from_location}-{to_location}/{formatted_date}?sort=bestflight_a'
    driver.get(url)

    
    wait_time = 20  # increase wait time if needed
    wait = WebDriverWait(driver, wait_time)

    while True:
        try:
            show_more_buttons = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.ULvh-button.show-more-button')))
            num_buttons = len(show_more_buttons)

            for i in range(num_buttons):
                show_more_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.ULvh-button.show-more-button')))
                driver.execute_script("arguments[0].click();", show_more_button)
                try:
                    wait.until(EC.staleness_of(show_more_button))
                except:
                    pass

            flight_rows = driver.find_elements(By.XPATH, '//div[@class="nrc6-inner"]')

        except TimeoutException:
            break
    
    return flight_rows
            

In [11]:
def convert_to_df(flight_rows):
    
    list_prices = []
    list_company_names=[]
    list_stops=[]
    list_duration=[]
    list_dates=[]
    
    for WebElement in flight_rows:
        elementHTML = WebElement.get_attribute('outerHTML')
        elementSoup = BeautifulSoup(elementHTML, 'html.parser')
        # price
        temp_price = elementSoup.find("div", {"class": "nrc6-price-section"})
        price = temp_price.find("div", {"class": "f8F1-price-text"})
        list_prices.append(price.text)

        #company names#
        company_names = elementSoup.find("div",{"class":"c_cgF c_cgF-mod-variant-default"}).text
        list_company_names.append(company_names)

        # stops
        temp_stops = elementSoup.find("div", {"class": "vmXl vmXl-mod-variant-default"})
        stops = temp_stops.find("span", {"class": "JWEO-stops-text"})
        list_stops.append(stops.text)

        #durations  
        temp_durations = elementSoup.find("div", {"class": "xdW8"})
        durations = temp_durations.find("div", {"class": "vmXl vmXl-mod-variant-default"}).text
        list_duration.append(durations)
    
    df = pd.DataFrame({
        'Price': list_prices,
        'Company Name': list_company_names,
        'Stops': list_stops,
        'Duration': list_duration,
        })

    df['Destination'] = to_location
    df['From'] = from_location
    df['Date'] = date
    
    return df


In [13]:
driver = webdriver.Chrome()

from_location = 'LAX'
to_location = 'HNL'

date = datetime.strptime('2023-06-01', '%Y-%m-%d')
end_date = datetime.strptime('2023-06-5', '%Y-%m-%d')
total_days = (end_date - date).days + 1

df = pd.DataFrame()
with tqdm(total=total_days) as pbar:

    while date <= end_date:

        df = pd.concat([df, convert_to_df(scrape_kayak(from_location, to_location, date))])
        
        date += timedelta(days=1)
        pbar.update(1)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [09:07<00:00, 109.42s/it]


In [14]:
df

,Price,Company Name,Stops,Duration,Destination,From,Date
0,$288,Alaska Airlines,1 stop,16h 25m,HNL,LAX,2023-06-01
1,$356,United Airlines,nonstop,5h 45m,HNL,LAX,2023-06-01
2,$288,Alaska Airlines,1 stop,16h 25m,HNL,LAX,2023-06-01
3,$366,United Airlines,nonstop,5h 46m,HNL,LAX,2023-06-01
4,$450,Hawaiian Airlines,nonstop,5h 50m,HNL,LAX,2023-06-01
...,...,...,...,...,...,...,...
356,"$2,546",American Airlines,1 stop,15h 13m,HNL,LAX,2023-06-05
357,Info,Southwest,nonstop,5h 50m,HNL,LAX,2023-06-05
358,Info,Southwest,1 stop,8h 05m,HNL,LAX,2023-06-05
359,Info,Southwest,1 stop,8h 20m,HNL,LAX,2023-06-05


In [15]:
df.to_csv('flight_LAX_HNL_data.csv', index=False)